In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime as dt

from swarmpal.io import PalDataItem, create_paldata
from swarmpal_mma.pal_processes import MMA_SHA_2E

## Set configurations

In [ ]:
# Global parameters (should always remain the same)
MODEL = "'Model' = 'CHAOS-Core' + 'CHAOS-Static'"
MODEL_WITH_IONO = "'Model' = 'CHAOS-Core' + 'CHAOS-Static' + 'MIO_SHA_2C'"
VIRES_AUXILIARIES = ["MLT", "QDLat", "Dst", "QDBasis", "DipoleAxisVector"]
AVAILABLE_DATASETS = {"Swarm-A": "SW_OPER_MAGA_LR_1B", "Swarm-B": "SW_OPER_MAGB_LR_1B"}
SAMPLING_STEP = "PT25S"

# Tunable parameters
PARAMS = {"LT_limit": 6, "min_gm_lat": 0, "max_gm_lat": 65}
# TODO: See what should be included here from MMA_2E/utils/Config.py

# Local settings (changes with each run) - these determine what external data is used
START_TIME = dt.datetime(2024, 10, 1)
END_TIME = dt.datetime(2024, 11, 1)
# START_TIME = dt.datetime(2024, 10, 1)
# END_TIME = dt.datetime(2024, 10, 2)
SPACECRAFTS_TO_USE = ["Swarm-A", "Swarm-B"]  # corresponds to AVAILABLE_DATASETS

## Fetch data using SwarmPAL

In [ ]:
def get_paldatatime_vires_config(
    collection=None, start_time=None, end_time=None, model=None
):
    """Set of options which are passed through to viresclient
    For options, see https://swarmpal.readthedocs.io/en/latest/api/io.html#swarmpal.io.PalDataItem.from_vires
    """
    return dict(
        collection=collection,
        measurements=["B_NEC"],
        models=[model],
        auxiliaries=VIRES_AUXILIARIES,
        sampling_step=SAMPLING_STEP,
        start_time=start_time,
        end_time=end_time,
        server_url="https://vires.services/ows",
        options=dict(asynchronous=True, show_progress=True),
    )


start_time = START_TIME
end_time = END_TIME
model = MODEL
data_config = {
    spacecraft: get_paldatatime_vires_config(
        collection=AVAILABLE_DATASETS.get(spacecraft),
        start_time=start_time,
        end_time=end_time,
        model=model,
    )
    for spacecraft in SPACECRAFTS_TO_USE
}
data_config

In [ ]:
data_in = create_paldata(
    **{
        label: PalDataItem.from_vires(**data_params)
        for label, data_params in data_config.items()
    }
)
print(data_in)

## Apply the MMA_SHA_2E process

In [ ]:
mma_process = MMA_SHA_2E()
mma_process.set_config()

In [ ]:
data_out = mma_process(data_in)

In [ ]:
print(data_out)

## Plotting

In [ ]:
from swarmpal_mma.Plotting.map_plot import map_surface_rtp

In [ ]:
data_out["MMA_SHA_2E"]["qs"][0, :]

In [ ]:
map_surface_rtp(data_out["MMA_SHA_2E"]["qs"][0, :])